# HCP_S3

In [1]:
from utils import hcp_s3

In [2]:

hcp_s3.list_buckets()

['hcp-openaccess',
 'hcp-openaccess-logfiles',
 'hcp-openaccess-logs-temp',
 'hcp-openaccess-logstorage-temp',
 'hcp-openaccess-test']

In [3]:
# List root
hcp_s3.ls(delimiter=True)

['HCP/',
 'HCP_1200/',
 'HCP_900/',
 'HCP_Resources/',
 'HCP_Retest/',
 'HCP_WB_Tutorial_1.0/']

In [4]:
# List a directory
hcp_s3.ls('HCP_1200/100610/T1w/fsaverage_LR32k/')

['HCP_1200/100610/T1w/fsaverage_LR32k/100610.32k_fs_LR.wb.spec',
 'HCP_1200/100610/T1w/fsaverage_LR32k/100610.L.inflated.32k_fs_LR.surf.gii',
 'HCP_1200/100610/T1w/fsaverage_LR32k/100610.L.inflated_MSMAll.32k_fs_LR.surf.gii',
 'HCP_1200/100610/T1w/fsaverage_LR32k/100610.L.midthickness.32k_fs_LR.surf.gii',
 'HCP_1200/100610/T1w/fsaverage_LR32k/100610.L.midthickness_MSMAll.32k_fs_LR.surf.gii',
 'HCP_1200/100610/T1w/fsaverage_LR32k/100610.L.midthickness_MSMAll_va.32k_fs_LR.shape.gii',
 'HCP_1200/100610/T1w/fsaverage_LR32k/100610.L.pial.32k_fs_LR.surf.gii',
 'HCP_1200/100610/T1w/fsaverage_LR32k/100610.L.pial_MSMAll.32k_fs_LR.surf.gii',
 'HCP_1200/100610/T1w/fsaverage_LR32k/100610.L.very_inflated.32k_fs_LR.surf.gii',
 'HCP_1200/100610/T1w/fsaverage_LR32k/100610.L.very_inflated_MSMAll.32k_fs_LR.surf.gii',
 'HCP_1200/100610/T1w/fsaverage_LR32k/100610.L.white.32k_fs_LR.surf.gii',
 'HCP_1200/100610/T1w/fsaverage_LR32k/100610.L.white_MSMAll.32k_fs_LR.surf.gii',
 'HCP_1200/100610/T1w/fsaverage_LR

In [5]:
# Download a file
hcp_s3.download('HCP_1200/100610/T1w/fsaverage_LR32k/100610.L.midthickness_MSMAll.32k_fs_LR.surf.gii', local='.temp')